In [99]:
import pandas as pd
import glob
import os
import numpy as np
from time import time
import logging
import gensim
import bz2
import PyPDF2
import requests
import json

In [100]:
dataset = []

path = '../datas/scrap/person/'
allFiles = glob.glob(os.path.join(path, '*.csv'))

# Load and concactenate all the files into one dataset.
for file_ in allFiles:
    data = pd.read_csv(file_)
    dataset += [data]
datas = pd.concat(dataset)

print(len(data.columns))  
#print('Length of the dataset', len(dataset))

22


In [101]:
datas[datas["FirstName"] == "Roger"]

,Unnamed: 0,DateOfBirth,DateOfDeath,FirstName,GenderAsString,ID,Language,LastName,MaritalStatus,MaritalStatusText,...,Modified,NativeLanguage,NumberOfChildren,OfficialName,PersonIdCode,PersonNumber,PlaceOfBirthCanton,PlaceOfBirthCity,Title,TitleText
564,564,1917-12-11T00:00:00,1998-02-13T00:00:00,Roger,m,641,FR,Schaffter,NaN,NaN,...,2015-05-17T21:18:19.387,NaN,NaN,NaN,1090.0,641,NaN,NaN,NaN,NaN
566,566,1914-12-02T00:00:00,1996-03-26T00:00:00,Roger,m,643,FR,Dafflon,NaN,NaN,...,2015-05-17T21:18:19.387,NaN,NaN,NaN,1092.0,643,NaN,NaN,NaN,NaN
176,176,1959-09-23T00:00:00,NaN,Roger,m,4137,FR,Golay,2.0,marié(e),...,2016-12-08T09:45:56.877,F,2.0,Golay,3039.0,4137,Genève,Genève,NaN,NaN
204,204,1965-03-21T00:00:00,NaN,Roger,m,4166,FR,Köppel,2.0,marié(e),...,2015-11-20T14:49:26.1,D,3.0,Köppel,3082.0,4166,Zurich,Zurich,12.0,lic. phil. I
73,73,1928-07-17T00:00:00,2005-06-29T00:00:00,Roger,m,1179,FR,Besuchet,NaN,NaN,...,2015-05-17T21:18:19.387,F,NaN,Besuchet,NaN,1179,NaN,NaN,NaN,NaN
115,115,1921-12-26T00:00:00,2008-11-18T00:00:00,Roger,m,1223,FR,Mugny,NaN,NaN,...,2015-05-17T21:18:19.387,F,NaN,Mugny Roger,NaN,1223,NaN,NaN,NaN,NaN
152,152,1973-03-23T00:00:00,NaN,Roger,m,1279,FR,Nordmann,2.0,marié(e),...,2015-11-13T14:49:35.527,F,2.0,Nordmann,2648.0,1279,Vaud,Lausanne,134.0,lic. ès sc. pol.
411,411,1907-09-12T00:00:00,1982-06-05T00:00:00,Roger,m,1593,FR,Bonvin,NaN,NaN,...,2015-05-17T21:18:19.387,F,NaN,Bonvin,NaN,1593,NaN,NaN,NaN,NaN


In [102]:
datas = datas.dropna(axis = 0, subset = ["PersonIdCode"])

In [103]:

ids = datas.PersonIdCode.unique().astype(int)


In [104]:
url_portrait = "https://www.parlament.ch/sitecollectionimages/profil/portrait-260/"
for id in ids:
    
    f = open('../datas/pictures/' + str(id) + '.jpg','wb')
    f.write(requests.get(url_portrait + str(id) + '.jpg').content)
    f.close()

In [109]:
datas[datas["FirstName"] == "Peter"]

,Unnamed: 0,DateOfBirth,DateOfDeath,FirstName,GenderAsString,ID,Language,LastName,MaritalStatus,MaritalStatusText,...,NativeLanguage,NumberOfChildren,OfficialName,PersonIdCode,PersonNumber,PlaceOfBirthCanton,PlaceOfBirthCity,Title,TitleText,id
6,6,1942-09-03T00:00:00,NaN,Peter,m,10,FR,Baumberger,2.0,marié(e),...,D,2.0,Baumberger Peter,2269.0,10,Zurich,Winterthur,6.0,Dr. iur.,Peter Baumberger
12,12,1939-06-01T00:00:00,NaN,Peter,m,16,FR,Bircher,2.0,marié(e),...,D,2.0,Bircher Peter,2260.0,16,Argovie,Wölflinswil,NaN,NaN,Peter Bircher
18,18,1933-09-16T00:00:00,NaN,Peter,m,22,FR,Bloetzer,1.0,célibataire,...,D,NaN,Bloetzer,2337.0,22,Valais,Brigue,95.0,dipl. Bauing. ETH,Peter Bloetzer
19,19,1952-03-30T00:00:00,NaN,Peter,m,23,FR,Bodenmann,NaN,NaN,...,NaN,NaN,Bodenmann Peter,2018.0,23,NaN,NaN,NaN,NaN,Peter Bodenmann
105,105,1948-05-14T00:00:00,NaN,Peter,m,111,FR,Hess,2.0,marié(e),...,D,4.0,Hess Peter,2085.0,111,Zoug,Unterägeri,6.0,Dr. iur.,Peter Hess
115,115,1957-09-26T00:00:00,NaN,Peter,m,121,FR,Jenni,NaN,NaN,...,D,NaN,Jenni,2302.0,121,NaN,NaN,NaN,NaN,Peter Jenni
190,190,1940-10-04T00:00:00,NaN,Peter,m,196,FR,Schmid,NaN,NaN,...,NaN,NaN,Schmid Peter,2161.0,196,NaN,NaN,NaN,NaN,Peter Schmid
218,218,1940-04-26T00:00:00,2009-07-04T00:00:00,Peter,m,225,FR,Tschopp,2.0,marié(e),...,F,2.0,Tschopp Peter,2329.0,225,Bâle-Ville,Bâle,91.0,Doctorat,Peter Tschopp
222,222,1946-04-14T00:00:00,NaN,Peter,m,229,FR,Vollmer,2.0,marié(e),...,D,3.0,Vollmer,2251.0,229,Berne,Berne,26.0,Dr. rer. pol.,Peter Vollmer
265,265,1952-06-21T00:00:00,NaN,Peter,m,277,FR,Bieri,2.0,marié(e),...,D,4.0,Bieri,2356.0,277,Zurich,Winterthur,226.0,Dipl. Ing. Agr. ETH / Dr. sc. techn.,Peter Bieri


In [110]:
datas['id'] = datas['FirstName']+' '+datas['LastName']
links = datas[["PersonIdCode"  ,"id"]]

In [111]:
links[links['id']== "Peter Schilliger"]

,PersonIdCode,id
164,3029.0,Peter Schilliger


In [112]:
dictionary = dict(zip(datas["id"].values.tolist(),datas["PersonIdCode"].astype(int).values.tolist()))

In [113]:
dictionary

{'Kurt Bürer': 1005,
 'Paul Wyss': 2191,
 'Serge Beck': 2461,
 'Jean-Claude Rennwald': 2413,
 'Flavio Cotti': 1082,
 'Hubert Bonvin': 2020,
 'Fritz Stalder': 2297,
 'Elisabeth Blunschy-Steiner': 1002,
 'Isabelle Chevalley': 3604,
 'Valentine Friedli': 1022,
 'Maximilian Reimann': 2147,
 'Theo Maissen': 2359,
 'Rudolf jun. Reichling': 2145,
 'Duri Campell': 3066,
 'Christoph Blocher': 2017,
 'Judith Stamm': 2173,
 'Hans Ulrich Baumberger': 1151,
 'Thomas Burgener': 2450,
 'Jean-Nils de Dardel': 2282,
 'Jean Christophe Schwaab': 3020,
 'Jacqueline Badran': 2762,
 'Christine Egerszegi-Obrist': 2380,
 'Werner Hösli': 3047,
 'Pierre Gassmann': 1153,
 'Lisa Mazzone': 3086,
 'Yvette Estermann': 2679,
 'Daniela Schneeberger': 2766,
 'Jean-Pierre Bonny': 2019,
 'Leo Müller': 3012,
 'Werner Carobbio': 2032,
 'Ursula Haller Vannini': 2496,
 'Walter Hess': 2499,
 'Walter Decurtins': 2482,
 'Marco Borradori': 2273,
 'Louis Schelbert': 2654,
 'Martina Munz': 3036,
 'Ernst Leuenberger': 2102,
 'Thoma

In [25]:
links.to_json("linkspersons.json")

In [107]:
with open('linkspersons.json', 'w') as outfile:
    json.dump(dictionary, outfile)